# Double check that sediment resuspension is working as intended

June, 2019

In [1]:
import os
os.environ['PROJ_LIB'] = '/home/brogalla/anaconda3/share/proj'

In [3]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import datetime
from pylab import *
import matplotlib
import warnings
import pickle
warnings.filterwarnings("ignore")

%matplotlib notebook

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [5]:
folder   = '/data/brogalla/run_storage/'
resus    = 'bdy-S-sed-nobio_2019-06-24T131116.040196-0400/'
no_resus = 'bdy-S-nosed-nobio_2019-06-24T131202.945597-0400/'
# resus    = 'sed-resus_2019-06-21T185829.455764-0400/'
# no_resus = 'sed-resus_2019-06-21T193443.986696-0400/'
file     = 'ANHA12_EXH006_5d_20020101_20021231_ptrc_T_20020101-20020105.nc'

Check fday files:

In [6]:
sed    = nc.Dataset(folder+resus+file)
no_sed = nc.Dataset(folder+no_resus+file)

sed2   = nc.Dataset(folder+'ANHA12_EXH006_5d_20020101_20021231_ptrc_T_20020814-20020818.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/data/brogalla/run_storage/bdy-S-sed-nobio_2019-06-24T131116.040196-0400/ANHA12_EXH006_5d_20020101_20021231_ptrc_T_20020101-20020105.nc'

In [14]:
rnf = np.array(sed.variables['runoff'])

KeyError: 'runoff'

In [ ]:
print(np.amax(rnf*1e-10), np.amin(rnf*1e-10), np.average(rnf*1e-10))

In [9]:
dmn_sed     = np.array(sed.variables['dissolmn'])
dmn_no_sed  = np.array(no_sed.variables['dissolmn'])
dmn_sed2    = np.array(sed2.variables['dissolmn'])

NameError: name 'sed' is not defined

In [10]:
diff_sed = dmn_sed-dmn_no_sed

NameError: name 'dmn_sed' is not defined

In [7]:
print(np.amax(diff_sed), np.amin(diff_sed), np.average(diff_sed))

NameError: name 'diff_sed' is not defined

In [8]:
print(np.amax(dmn_sed), np.amin(dmn_sed))
print(np.amax(dmn_no_sed), np.amin(dmn_no_sed))

NameError: name 'dmn_sed' is not defined

In [4]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])
tmask = tmask[0,:,1480:2050,160:800]

In [5]:
ref   = nc.Dataset('/data/brogalla/ANHA12/2015/ANHA12-EXH006_y2015m01d05_gridT.nc')
lons  = np.array(ref.variables['nav_lon'])
lats  = np.array(ref.variables['nav_lat'])
depth = np.array(ref.variables['deptht'])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(8,6))
sed = np.ma.masked_where((tmask[:,287,200] < 0.1), dmn_sed[0,:,287,200]) 
ax[0].plot(sed, depth)
ax[0].invert_yaxis()
ax[0].set_ylabel('depth')
ax[0].set_xlabel('dMn')
ax[0].set_title('Lon: '+ str(lons[1767,380]) + ', Lat: ' + str(lats[1767,380]))
ax[0].set_xlim([0, 6e-9])
#1777

sed2 = np.ma.masked_where((tmask[:,495,500] < 0.1), dmn_sed[0,:,495,500]) 
ax[1].plot(sed2, depth)
ax[1].invert_yaxis()
ax[1].set_xlabel('dMn')
ax[1].set_title('Lon: '+ str(lons[1977,680]) + ', Lat: ' + str(lats[1975,680]))
ax[1].set_xlim([0, 6e-9])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(8,6))
sed = np.ma.masked_where((tmask[:,287,220] < 0.1), dmn_sed2[0,:,287,220]) 
ax[0].plot(sed, depth)
ax[0].invert_yaxis()
ax[0].set_ylabel('depth')
ax[0].set_xlabel('dMn')
ax[0].set_title('Lon: '+ str(lons[1767,380]) + ', Lat: ' + str(lats[1767,380]))
ax[0].set_xlim([0, 1.5e-8])
#1777

sed2 = np.ma.masked_where((tmask[:,495,520] < 0.1), dmn_sed2[0,:,495,520]) 
ax[1].plot(sed2, depth)
ax[1].invert_yaxis()
ax[1].set_xlabel('dMn')
ax[1].set_title('Lon: '+ str(lons[1977,680]) + ', Lat: ' + str(lats[1975,680]))
ax[1].set_xlim([0, 1.5e-8])

In [ ]:
proj = pickle.load(open('/ocean/brogalla/GEOTRACES/analysis-brogalla/maps/surface-Mn.pickle','rb'))

x, y = proj(lons[1480:2050,160:800], lats[1480:2050,160:800])

var = np.ma.masked_where((tmask[0,:,:] < 0.1), dmn_sed2[0,0,:,:]) 

# clevels=[0, 0.25e-9, 0.5e-9, 0.75e-9, 1e-9, 1.25e-9, 1.5e-9, 1.75e-9, 2e-9, \
#          2.25e-9, 2.5e-9, 2.75e-9, 3e-9, 3.25e-9, 3.5e-9, 3.75e-9, 4e-9]
cbar = proj.contourf(x, y, var, cmap=cmocean.cm.thermal, zorder=1)

x_star, y_star = proj(lons[1767,380], lats[1767,380])
proj.plot(x_star, y_star, '*', markersize=15, c='y', zorder=3)

x_star1, y_star1 = proj(lons[1975,680], lats[1975,680])
proj.plot(x_star1, y_star1, '*', markersize=15, c='y', zorder=3)


fig = gcf()
cbar_ax = fig.add_axes([0.91, 0.15, 0.04, 0.7])
fig.colorbar(cbar, cax=cbar_ax);

In [ ]:
proj = pickle.load(open('/ocean/brogalla/GEOTRACES/analysis-brogalla/maps/surface-Mn.pickle','rb'))

x, y = proj(lons[1480:2050,180:820], lats[1480:2050,180:820])

var = np.ma.masked_where((tmask[0,:,:] < 0.1), diff_sed[0,0,:,:]) 

# clevels=[0, 0.25e-9, 0.5e-9, 0.75e-9, 1e-9, 1.25e-9, 1.5e-9, 1.75e-9, 2e-9, \
#          2.25e-9, 2.5e-9, 2.75e-9, 3e-9, 3.25e-9, 3.5e-9, 3.75e-9, 4e-9]
cbar = proj.contourf(x, y, var, cmap=cmocean.cm.thermal,vmin=0, vmax=2.8e-10, zorder=1)

x_star, y_star = proj(lons[1767,380], lats[1767,380])
proj.plot(x_star, y_star, '*', markersize=15, c='y', zorder=3)

x_star1, y_star1 = proj(lons[1975,680], lats[1975,680])
proj.plot(x_star1, y_star1, '*', markersize=15, c='y', zorder=3)

plt.title('Difference between run with zeroed trb and run without')

fig = gcf()
cbar_ax = fig.add_axes([0.91, 0.15, 0.04, 0.7])
fig.colorbar(cbar, cax=cbar_ax);

In [ ]:
proj = pickle.load(open('/ocean/brogalla/GEOTRACES/analysis-brogalla/maps/surface-Mn.pickle','rb'))

x, y = proj(lons[1480:2050,180:820], lats[1480:2050,180:820])

var = np.ma.masked_where((tmask[0,:,:] < 0.1), rnf[0,0,:,:]*1e-10) 

clevels1=[0, 5, 10, 15, 20, 25, 30]
cbar = proj.contourf(x, y, var, cmap=cmocean.cm.thermal, clevels=clevels1, zorder=1)

fig = gcf()
cbar_ax = fig.add_axes([0.91, 0.15, 0.04, 0.7])
fig.colorbar(cbar, cax=cbar_ax);